In [1]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import logging
import json

# Add the src directory to sys.path
path = str(Path().resolve().parent)
if path not in sys.path:
    sys.path.insert(0, path)

print(sys.path)

# load_dotenv()
# print(str(dict(os.environ.items())))


LOGGING = {
    "version": 1,
    "disable_existing_loggers": False,
    "formatters": {
        "json": {
            "()": "pythonjsonlogger.jsonlogger.JsonFormatter",
            "fmt": "%(asctime)s %(levelname)s %(name)s %(message)s",

        }
    },
}


# Set up structured logging to a file. All of the cells in this notebook use
# this logger. Find them at .data/logs.jsonl.
class JsonFormatter(logging.Formatter):
    def format(self, record) -> str:
        record_dict = record.__dict__
        log_record = {
            'timestamp': self.formatTime(record, self.datefmt),
            'level': record.levelname,
            'session_id': record_dict.get('session_id', None),
            'run_id': record_dict.get('run_id', None),
            'message': record.getMessage(),
            'data': record_dict.get('data', None),
            'module': record.module,
            'funcName': record.funcName,
            'lineNumber': record.lineno,
            'logger': record.name,
        }
        extra_fields = {
            key: value for key, value in record.__dict__.items() 
            if key not in ['levelname', 'msg', 'args', 'exc_info', 'funcName', 'module', 'lineno', 'name', 'message', 'asctime', 'session_id', 'run_id', 'data']
        }
        log_record.update(extra_fields)
        return json.dumps(log_record)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)
if logger.hasHandlers():
    logger.handlers.clear()
data_dir = Path('.data')
if not data_dir.exists():
    data_dir.mkdir()
handler = logging.FileHandler(data_dir / 'logs.jsonl')
handler.setFormatter(JsonFormatter())
logger.addHandler(handler)


['/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/workspaces/semanticworkbench/libraries/python/skills/skills/form-filler-skill/.venv/lib/python3.11/site-packages', '/workspaces/semanticworkbench/libraries/python/assistant-drive', '/workspaces/semanticworkbench/libraries/python/context', '/workspaces/semanticworkbench/libraries/python/events', '/workspaces/semanticworkbench/libraries/python/skills/skills/form-filler-skill', '/workspaces/semanticworkbench/libraries/python/openai-client', '/workspaces/semanticworkbench/libraries/python/semantic-workbench-api-model', '/workspaces/semanticworkbench/libraries/python/semantic-workbench-assistant', '/workspaces/semanticworkbench/libraries/python/skills/skill-library']


In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
from textwrap import dedent

from assistant_drive import Drive, DriveConfig
from form_filler_skill.guided_conversation.definition import GCDefinition
from form_filler_skill.guided_conversation import GuidedConversationSkill, TUnanswered, UNANSWERED
from form_filler_skill.guided_conversation.resources import (
    ResourceConstraint,
    ResourceConstraintMode,
    ResourceConstraintUnit,
)
from openai_client import AzureOpenAIAzureIdentityAuthConfig, AzureOpenAIServiceConfig, create_client
from pydantic import BaseModel, Field, HttpUrl
from skill_library import Skill
from skill_library.routine_stack import RoutineStack
from skill_library.run_context import LogEmitter, RunContext
from skill_library.skill_registry import SkillRegistry
from openai_client.chat_driver import ChatDriverConfig
from skill_library import Assistant
import asyncio

service_config = AzureOpenAIServiceConfig(
    auth_config=AzureOpenAIAzureIdentityAuthConfig(),
    azure_openai_endpoint=HttpUrl("https://lightspeed-team-shared-openai-eastus.openai.azure.com/"),
    azure_openai_deployment="gpt-4o",
)
language_model = create_client(service_config)

class Artifact(BaseModel):
    student_poem: str | TUnanswered = Field(description="The acrostic poem written by the student.", default=UNANSWERED)
    initial_feedback: str| TUnanswered = Field(description="Feedback on the student's final revised poem.", default=UNANSWERED)
    final_feedback: str| TUnanswered = Field(description="Feedback on how the student was able to improve their poem.", default=UNANSWERED)
    inappropriate_behavior: list[str]| TUnanswered = Field(
        description=dedent("""
            List any inappropriate behavior the student attempted while chatting with you.
            It is ok to leave this field Unanswered if there was none.
        """),
        default=UNANSWERED
    )

# artifact_schema = Artifact.model_json_schema()
resource_constraint = ResourceConstraint(
        quantity=10,
        unit=ResourceConstraintUnit.TURNS,
        mode=ResourceConstraintMode.EXACT,
    )
definition = GCDefinition(
    artifact_schema=Artifact.model_json_schema(),
    rules=[
        "DO NOT write the poem for the student.",
        "Terminate the conversation immediately if the students asks for harmful or inappropriate content.",
    ],
    conversation_flow=dedent("""
        1. Start by explaining interactively what an acrostic poem is.
        2. Then give the following instructions for how to go ahead and write one:
            1. Choose a word or phrase that will be the subject of your acrostic poem.
            2. Write the letters of your chosen word or phrase vertically down the page.
            3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
            4. Write these words or phrases next to the corresponding letters to create your acrostic poem.
        3. Then give the following example of a poem where the word or phrase is HAPPY:
            Having fun with friends all day,
            Awesome games that we all play.
            Pizza parties on the weekend,
            Puppies we bend down to tend,
            Yelling yay when we win the game
        4. Finally have the student write their own acrostic poem using the word or phrase of their choice. Encourage them
        to be creative and have fun with it. After they write it, you should review it and give them feedback on what they
        did well and what they could improve on. Have them revise their poem based on your feedback and then review it again.
    """),
    conversation_context=dedent("""
        You are working 1 on 1 a 4th grade student who is chatting with you in the computer lab at school while being
        supervised by their teacher.
    """),
    resource_constraint=resource_constraint,
)

# Configure the skill and its dependencies.
path = Path(".data/assistant-1")
drive = Drive(DriveConfig(root=path))
skill = GuidedConversationSkill(name="guided_conversation", language_model=language_model, definition=definition, drive=drive.subdrive("gc1"))
skill_dependency_tree: dict[str, Skill] = {skill.name: skill}

# Set up a run context.
routine_stack = RoutineStack(drive)
skill_registry = SkillRegistry(skill_dependency_tree, routine_stack)
runContext = RunContext(
    "test-session-1", drive, LogEmitter().emit, routine_stack, skill_registry.run_routine_by_designation
)

# Define the assistant.
chat_driver_config = ChatDriverConfig(
    openai_client=language_model,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
)

assistant = Assistant(name="Alice", assistant_id="gc-assistant-123", chat_driver_config=chat_driver_config, skills=skill_dependency_tree, startup_routine="guided_conversation.conversation")
await assistant.clear()
await assistant.start()

# Run the skill with it's context.
# if await routine_stack.peek() is None:
#     await routine_stack.push("current_frame")
#     await skill.conversation_init_function(context=runContext, vars={"artifact": Artifact()})

# Function that allows user input in a non-blocking manner.
async def user_input_handler() -> None:
    while True:
        user_input = await asyncio.to_thread(input, "User: ")
        if user_input == "":
            assistant.stop()
            break
        print(f"User: {user_input}", flush=True)
        await assistant.put_message(user_input)


# Start the user input in a non-blocking way.
input_task = asyncio.create_task(user_input_handler())

# Start the assistant.
async for event in assistant.events:
    print(f"Assistant: {event.message}", flush=True)  # type: ignore

await assistant.wait()
await input_task



Assistant: Agenda updated
Assistant: Hello! Let's dive into creating an acrostic poem together. An acrostic poem is a fun type of poem where the first letters of each line spell out a word or a message vertically. Here's how you can make one:

1. Choose a word or phrase that will be the subject of your acrostic poem.
2. Write the letters of your chosen word or phrase vertically down the page.
3. Think of a word or phrase that starts with each letter of your chosen word or phrase.
4. Write these words or phrases next to the corresponding letters to create your acrostic poem.

For example, if you choose the word "HAPPY," it might look like this:

- **H**aving fun with friends all day,
- **A**wesome games that we all play.
- **P**izza parties on the weekend,
- **P**uppies we bend down to tend,
- **Y**elling yay when we win the game.

Now, it's your turn! Pick a word or phrase you love, and jot down an acrostic poem. Don't forget to have fun and be creative! When you're ready, share it wit